In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -U \
  transformers \
  datasets \
  peft \
  accelerate \
  trl \
  sentencepiece


In [3]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer


In [4]:
dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/instructions.jsonl",
    split="train"
)


In [5]:

model_id = "microsoft/Phi-3-mini-4k-instruct"


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# 🔑 CRITICAL LINE (memory control)
tokenizer.model_max_length = 1024

# Load model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Enable gradient checkpointing

In [7]:
model.gradient_checkpointing_enable()
model.enable_input_require_grads()


In [8]:
def formatting_func(example):
    return f"""### Instruction:
{example['instruction']}

### Response:
{example['output']}"""

LoRA Config

In [9]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["qkv_proj", "out_proj"],  # Phi-safe
)

Training Arguments

In [10]:
training_args = TrainingArguments(
    output_dir="./lora-phi3",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    num_train_epochs=2,
    fp16=True,
    bf16=False,
    logging_steps=50,
    save_strategy="epoch",
    optim="adamw_torch",   # ✅ IMPORTANT
    report_to="none",
)

Trainer Init

In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=training_args,
    formatting_func=formatting_func,
    processing_class=tokenizer,
)

Applying formatting function to train dataset:   0%|          | 0/12000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/12000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/12000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1665 > 1024). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/12000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [12]:
import torch
torch.cuda.empty_cache()


TRAIN

In [13]:
trainer.train()

Step,Training Loss
50,1.254800
100,1.167900
150,1.127100
200,1.184800
250,1.166600
300,1.167900
350,1.144800
400,1.145300
450,1.142900
500,1.119800


TrainOutput(global_step=1500, training_loss=1.1386388041178386, metrics={'train_runtime': 8242.6644, 'train_samples_per_second': 2.912, 'train_steps_per_second': 0.182, 'total_flos': 8.959028065567949e+16, 'train_loss': 1.1386388041178386, 'epoch': 2.0})

Save Adapter

In [15]:
trainer.model.save_pretrained("/content/drive/MyDrive/phi3_lora_final")
tokenizer.save_pretrained("/content/drive/MyDrive/phi3_lora_final")


('/content/drive/MyDrive/phi3_lora_final/tokenizer_config.json',
 '/content/drive/MyDrive/phi3_lora_final/special_tokens_map.json',
 '/content/drive/MyDrive/phi3_lora_final/chat_template.jinja',
 '/content/drive/MyDrive/phi3_lora_final/tokenizer.model',
 '/content/drive/MyDrive/phi3_lora_final/added_tokens.json',
 '/content/drive/MyDrive/phi3_lora_final/tokenizer.json')

In [18]:
!zip -r lora-phi3-final.zip /content/drive/MyDrive/phi3_lora_final


  adding: content/drive/MyDrive/phi3_lora_final/ (stored 0%)
  adding: content/drive/MyDrive/phi3_lora_final/README.md (deflated 65%)
  adding: content/drive/MyDrive/phi3_lora_final/adapter_model.safetensors (deflated 7%)
  adding: content/drive/MyDrive/phi3_lora_final/adapter_config.json (deflated 56%)
  adding: content/drive/MyDrive/phi3_lora_final/chat_template.jinja (deflated 60%)
  adding: content/drive/MyDrive/phi3_lora_final/tokenizer_config.json (deflated 86%)
  adding: content/drive/MyDrive/phi3_lora_final/special_tokens_map.json (deflated 73%)
  adding: content/drive/MyDrive/phi3_lora_final/added_tokens.json (deflated 62%)
  adding: content/drive/MyDrive/phi3_lora_final/tokenizer.model (deflated 55%)
  adding: content/drive/MyDrive/phi3_lora_final/tokenizer.json (deflated 85%)
